In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('./preprocessing')
sys.path.append('./seq2seq2')

In [3]:
from processor import Code_Intent_Pairs, sub_slotmap
from model import Seq2Seq
from data import get_train_loader, get_test_loader, write_answer_json

### Define Hyperparameters

In [25]:
hyperP = {
    ## training parameters
    'batch_size' : 32,
    'lr' : 1e-3,
    'teacher_force_rate' : 0.90,
    'max_epochs' : 50,
    'lr_keep_rate' : 0.95,  # set to 1.0 to not decrease lr overtime
    
    ## encoder architecture
    'encoder_layers' : 2,
    'encoder_embed_size' : 128,
    'encoder_hidden_size' : 384,
    'encoder_dropout_rate' : 0.3,
    
    ## decoder architecture
    'decoder_layers' : 2,
    'decoder_embed_size' : 128,
    'decoder_hidden_size' : 384,
    'decoder_dropout_rate' : 0.3,
    
    ## attn architecture
    'attn_hidden_size' : 384,
    
    ## visualization
    'print_every': 10,
}

### Load Data

In [5]:
code_intent_pair = Code_Intent_Pairs()

In [6]:
path = 'vocab/'
code_intent_pair.load_dict(path)
special_symbols = code_intent_pair.get_special_symbols()
word_size = code_intent_pair.get_word_size()
code_size = code_intent_pair.get_code_size()

In [7]:
train_path = 'processed_corpus/train.json'
train_entries = code_intent_pair.load_entries(train_path)
code_intent_pair.pad()

In [8]:
trainloader = get_train_loader(train_entries, special_symbols, hyperP)

In [9]:
valid_path = 'processed_corpus/valid.json'
valid_entries = code_intent_pair.load_entries(valid_path)
code_intent_pair.pad()

In [10]:
validloader = get_train_loader(valid_entries, special_symbols, hyperP)

In [11]:
test_path = 'processed_corpus/test.json'
test_entries = code_intent_pair.load_entries(test_path)

In [12]:
testloader = get_test_loader(test_entries)

### Define Model

In [13]:
model = Seq2Seq(word_size, code_size, hyperP)

### Training

In [14]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
optimizer = optim.Adam(model.parameters(), lr=hyperP['lr'])
loss_f = torch.nn.CrossEntropyLoss()

In [15]:
lr_keep_rate = hyperP['lr_keep_rate']
if lr_keep_rate != 1.0:
    lr_reduce_f = lambda epoch: lr_keep_rate ** epoch
    scheduler = LambdaLR(optimizer, lr_lambda=lr_reduce_f)

In [26]:
def train(model, trainloader, optimizer, loss_f, hyperP):
    model.train()
    loss_sum = 0
    total_correct = 0
    size = 0
    print_every = hyperP['print_every']
    
    for i, (inp_seq, original_out_seq, padded_out_seq, out_lens) in enumerate(trainloader):
        logits = model(inp_seq, padded_out_seq, out_lens)
        loss = loss_f(logits, original_out_seq)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # show stats
        loss_sum += loss.item()
        _, predictions = torch.max(logits, dim=1)
        total_correct += (predictions == original_out_seq).sum()
        size += len(original_out_seq)

        if (i+1) % print_every == 0:
            print('Train: loss:{}\tacc:{}'.format(loss_sum/print_every, float(total_correct)/size))
            loss_sum = 0
            total_correct = 0
            size = 0
    print()

In [27]:
def valid(model, validloader, loss_f, hyperP):
    model.eval()
    old_rate = model.change_teacher_force_rate(0.0)
    loss_sum = 0
    total_correct = 0
    size = 0
    print_every = hyperP['print_every']
    
    with torch.no_grad():
        for i, (inp_seq, original_out_seq, padded_out_seq, out_lens) in enumerate(validloader):
            logits = model(inp_seq, padded_out_seq, out_lens)
            loss = loss_f(logits, original_out_seq)

            # show stats
            loss_sum += loss.item()
            _, predictions = torch.max(logits, dim=1)
            total_correct += (predictions == original_out_seq).sum()
            size += len(original_out_seq)

    print('Valid: loss:{}\tacc:{}'.format(loss_sum/len(validloader), float(total_correct)/size))
    model.change_teacher_force_rate(old_rate)
    return float(total_correct)/size

In [28]:
best_acc = 0.0

In [62]:
for e in range(hyperP['max_epochs']):
    train(model, trainloader, optimizer, loss_f, hyperP)
    acc = valid(model, validloader, loss_f, hyperP)
    if acc > best_acc:
        best_acc = acc
        model.save()
        print('model saved')
    if lr_keep_rate != 1.0:
        scheduler.step()

Train: loss:0.18472503498196602	acc:0.9559518693596906


Train: loss:0.2579076781868935	acc:0.9315557421792987


Train: loss:0.18371288627386093	acc:0.9554521276595744


Train: loss:0.19043032974004745	acc:0.9559371121224659


Train: loss:0.19171097204089166	acc:0.9508786788058438


Train: loss:0.17079838663339614	acc:0.9563519411899839


Train: loss:0.17505916729569435	acc:0.9588051072030835


Valid: loss:8.07654783129692	acc:0.2205121401087966


Train: loss:0.1698269799351692	acc:0.9604641168886979


KeyboardInterrupt: 

In [61]:
model.load()

### Decoding

In [63]:
sos = special_symbols['code_sos']
eos = special_symbols['code_eos']
unk = special_symbols['code_unk']
code_list = []
for i, (src_seq, slot_map, code, intent) in enumerate(testloader):
    model.eval()
    seq = model.beam_decode(src_seq, sos, eos, unk, beam_width=13)
    gen_code_tokens = code_intent_pair.idx2code(seq)
    gen_code = sub_slotmap(gen_code_tokens, slot_map)
    code_list.append(gen_code)
    
    # seq = model.greedy_decode(src_seq, sos, eos, unk)
    # gen_code_tokens = code_intent_pair.idx2code(seq)
    # gen_code = sub_slotmap(gen_code_tokens, slot_map)
    # code_list.append(gen_code)
    print('intent:\t'+intent)
#     print('predicted_beams:\t'+gen_code)
    print('predicted_greed:\t'+gen_code)
    print('ground_truth:   \t'+code)
    print()
    
#     if i == 5:
#         break

intent:	send a signal `signal.SIGUSR1` to the current process
predicted_greed:	sys.encode('signal.SIGUSR1' , 1000.encode('signal.SIGUSR1'))
ground_truth:   	os.kill(os.getpid(), signal.SIGUSR1)



intent:	decode a hex string '4a4b4c' to UTF-8.
predicted_greed:	"""4a4b4c""".encode('hex')
ground_truth:   	bytes.fromhex('4a4b4c').decode('utf-8')



intent:	check if all elements in list `myList` are identical
predicted_greed:	all(isinstance(x , int) for x in myList)
ground_truth:   	all(x == myList[0] for x in myList)



intent:	format number of spaces between strings `Python`, `:` and `Very Good` to be `20`
predicted_greed:	print(Python.format(Python).split('Very Good'))
ground_truth:   	print('%*s : %*s' % (20, 'Python', 20, 'Very Good'))

intent:	How to convert a string from CP-1251 to UTF-8?
predicted_greed:	s.encode('utf-8')
ground_truth:   	d.decode('cp1251').encode('utf8')



intent:	get rid of None values in dictionary `kwargs`
predicted_greed:	next(iter(list(kwargs.values()) , key = lambda x : x [ 1 ])
ground_truth:   	res = {k: v for k, v in list(kwargs.items()) if v is not None}



...

intent:	choose a random file from the directory contents of the C drive, `C:\\`
predicted_greed:	os.system('C:\\')
ground_truth:   	random.choice(os.listdir('C:\\'))



intent:	get the highest element in absolute value in a numpy matrix `x`
predicted_greed:	np.x(np.x(x))
ground_truth:   	max(x.min(), x.max(), key=abs)

intent:	Get all urls within text `s`
predicted_greed:	print(s.<eos>)
ground_truth:   	re.findall('"(http.*?)"', s, re.MULTILINE | re.DOTALL)



intent:	match urls whose domain doesn't start with `t` from string `document` using regex
predicted_greed:	re.findall(document , t)
ground_truth:   	re.findall('http://[^t][^s"]+\\.html', document)

intent:	split a string `mystring` considering the spaces ' '
predicted_greed:	re.split('' , mystring)
ground_truth:   	mystring.replace(' ', '! !').split('!')



intent:	open file `path` with mode 'r'
predicted_greed:	path = open('r' , os.path | 'r')
ground_truth:   	open(path, 'r')



intent:	sum elements at the same index in list `data`
predicted_greed:	map(sum , zip(* data))
ground_truth:   	[[sum(item) for item in zip(*items)] for items in zip(*data)]

intent:	add a new axis to array `a`
predicted_greed:	a.append()
ground_truth:   	a[:, (np.newaxis)]



In [64]:
write_answer_json(code_list)

In [65]:
token_list = ['all','.', '(', 'isinstance', '(', 'word' ',', 'myList',')', ')' ,'==','0']
sub_slotmap(token_list, {0:"0"})

'all.(isinstance(word, myList)) == 0'

In [66]:
model.save()